In [62]:
import numpy as np

from numba import njit, jit

import sys
sys.path.append('..')
from lib import *
from lib.maxent import *

In [90]:
def mcmcsampler(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1, prng=None):
    """Markov chain Monte carlo sampler.

    x0: starting position (array)
    energy(x): function for calculating energy
    jump(x): function for calculating a proposed new position
    nburnin: burnin period in which states are not saved
    nsample: sample interval for saving states
    
    returns array of states
    """
    if prng is None:
        prng = np.random
    nsteps, nburnin, nsample = int(nsteps), int(nburnin), int(nsample)
    x = x0
    Ex = energy(x)
    samples = np.zeros(((nsteps-nburnin)//nsample, x0.shape[0]), dtype=np.int64)
    counter = 0
    for i in range(1, nsteps+1):
        xp = jump(x)
        Exp = energy(xp)
        if (Exp < Ex) or (prng.rand() < np.exp(-Exp+Ex)):
            x = xp
            Ex = Exp
        if (i > nburnin) and (i % nsample == 0):
            samples[counter] = x
            counter += 1
    return samples

In [91]:
prng = np.random
L = 9
q = naminoacids
jump = lambda x: local_jump_jit(x, q)
x0 = prng.randint(q, size=L)
nsteps = 10000
nsample = 9

In [92]:
params = np.load('data/Human_full_k9.npz')
hi = params['hi']
Jij = params['Jij']
energy = lambda x: energy_potts(x, hi, Jij)

In [93]:
mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample);

In [95]:
%timeit mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample)

26.8 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
%prun mcmcsampler(x0, energy, jump, nsteps=nsteps, nsample=nsample);

In [86]:
@njit
def mcmcsampler_jit(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1):
    """Markov chain Monte carlo sampler.

    x0: starting position (array)
    energy(x): function for calculating energy
    jump(x): function for calculating a proposed new position
    nburnin: burnin period in which states are not saved
    nsample: sample interval for saving states
    
    returns array of states
    """
    prng = np.random
    nsteps, nburnin, nsample = int(nsteps), int(nburnin), int(nsample)
    x = x0
    Ex = energy(x)
    samples = np.zeros(((nsteps-nburnin)//nsample, x0.shape[0]), dtype=np.int64)
    counter = 0
    for i in range(1, nsteps+1):
        xp = jump(x)
        Exp = energy(xp)
        if (Exp < Ex) or (prng.rand() < np.exp(-Exp+Ex)):
            x = xp
            Ex = Exp
        if (i > nburnin) and (i % nsample == 0):
            samples[counter] = x
            counter += 1
    return samples

In [87]:
@njit
def energy(x):
    return energy_potts(x, hi, Jij)

@njit
def jump(x):
    return local_jump_jit(x, q)

In [88]:
mcmcsampler_jit(x0, energy, jump, nsteps=nsteps, nsample=nsample);

In [89]:
%timeit mcmcsampler_jit(x0, energy, jump, nsteps=nsteps, nsample=nsample)

2.12 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
